<a href="https://colab.research.google.com/github/dornercr/INFO371/blob/main/INFO371_Evaluation_Scaling_Dimensionality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INFO 371: Data Mining Applications
___

### Week 3: Evaluation, Scaling, and Dimensionality

### Charles Dorner, EdD (Candidate)
### College of Computing and Informatics, Drexel University

In [1]:
# === 1. Import Libraries ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files